# Time to slice & dice

## Load dataset

In [1]:
from datasets import load_dataset

In [2]:
squad = load_dataset("squad", split="train")

In [3]:
squad[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

## Shuffle

In [4]:
squad_shuffled = squad.shuffle(seed=666)
squad_shuffled[0]

{'id': '5727cc873acd2414000deca9',
 'title': 'Oklahoma',
 'context': 'Oklahoma is the 20th largest state in the United States, covering an area of 69,898 square miles (181,035 km2), with 68,667 square miles (177847 km2) of land and 1,281 square miles (3,188 km2) of water. It is one of six states on the Frontier Strip and lies partly in the Great Plains near the geographical center of the 48 contiguous states. It is bounded on the east by Arkansas and Missouri, on the north by Kansas, on the northwest by Colorado, on the far west by New Mexico, and on the south and near-west by Texas.',
 'question': 'Where does Oklahoma rank by land area?',
 'answers': {'text': ['20th'], 'answer_start': [16]}}

In [5]:
dataset = squad.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 78839
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 8760
    })
})

## Select

In [6]:
indices = [0, 10, 20, 40, 80]
examples = squad.select(indices)
examples

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5
})

In [7]:
sample = squad.shuffle().select(range(5)); sample

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5
})

## Filter

In [8]:
squad_filtered = squad.filter(lambda x: x["title"].startswith("L"))
squad_filtered

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2049
})

## Rename

In [9]:
squad.rename_column("context", "passages")

Dataset({
    features: ['id', 'title', 'passages', 'question', 'answers'],
    num_rows: 87599
})

## Remove

In [10]:
squad.remove_columns(["id", "title"])

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 87599
})

## Flatten

In [11]:
squad.flatten()

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers.text', 'answers.answer_start'],
    num_rows: 87599
})

## Map

In [12]:
def lowercase_title(example): return {'title': example["title"].lower()}

In [13]:
squad_lowercase = squad.map(lowercase_title)

In [14]:
squad_lowercase.shuffle(seed=42)["title"][:5]

['egypt',
 'ann_arbor,_michigan',
 'rule_of_law',
 'samurai',
 'group_(mathematics)']

## Batched Map

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_title(example): return tokenizer(example["title"])

squad.map(tokenize_title, batched=True, batch_size=500)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask'],
    num_rows: 87599
})

## Drug Review Dataset

In [16]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2023-09-14 14:05:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip.1’

drugsCom_raw.zip.1      [         <=>        ]  41.00M  6.90MB/s    in 12s     

2023-09-14 14:06:10 (3.52 MB/s) - ‘drugsCom_raw.zip.1’ saved [42989872]

Archive:  drugsCom_raw.zip
replace drugsComTest_raw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [17]:
data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}

In [18]:
drug_dataset = load_dataset("csv", data_files=data_files, delimiter = "\t")

In [19]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [20]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

In [21]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [22]:
drug_dataset = drug_dataset.rename_column(
    original_column_name = "Unnamed: 0", new_column_name="patient_id"
)

In [23]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [24]:
for split in drug_dataset.keys():
    print(split)
    print("Number of drugs ", len(drug_dataset[split].unique("drugName")))
    print("Number of conditions ", len(drug_dataset[split].unique("condition")))

train
Number of drugs  3436
Number of conditions  885
test
Number of drugs  2637
Number of conditions  709


In [25]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}


In [26]:
def filter_nones(x):
    return x["condition"] is not None

In [27]:
drug_dataset.filter(filter_nones)

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 160398
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53471
    })
})

In [28]:
(lambda x: x * x)(3)

9

In [29]:
(lambda base, height: 0.5 * height * base)(4, 8)

16.0

In [30]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

In [31]:
drug_dataset = drug_dataset.map(lowercase_condition)

In [32]:
drug_dataset["train"]["condition"][:3]

['left ventricular dysfunction', 'adhd', 'birth control']

In [33]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

In [34]:
drug_dataset = drug_dataset.map(compute_review_length)

In [35]:
drug_dataset["train"][0]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [36]:
drug_dataset["train"].sort("review_length")[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [37]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)

In [38]:
drug_dataset.num_rows

{'train': 138514, 'test': 46108}

In [41]:
sorted_drug_dataset = drug_dataset['test'].sort('review_length', reverse=True)

In [45]:
sorted_drug_dataset[:1]

{'patient_id': [45000],
 'drugName': ['Fluoxetine'],
 'condition': ['obsessive compulsive disorde'],
 'review': ['"I don&rsquo;t find a lot of positive stories about antidepressants, or I find stories where people are taking the antidepressant the wrong way.\r\n\r\nI wanted to share my experience.  A positive one.\r\n\r\nI&rsquo;ve had generalized anxiety disorder, SEVERE OCD, and panic disorder for as long as I can remember.  My first memory of having an episode was when I was 4 years old at my kindergarten interview.  I feel as though I was born with the illnesses mentioned above, right from the womb.  When I was a child I was extremely anxious, had bad separation anxiety from my parents and had extreme OCD, I was just a kid and thought that the way I was feeling is how all kids felt, I didn&rsquo;t realize that I was different.  This went on, and got even worse in middle school.  I began developing trichtilomania in middle school.  In high school I went from being a 90% above studen

In [46]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [47]:
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [49]:
drug_dataset['test'][:1]

{'patient_id': [163740],
 'drugName': ['Mirtazapine'],
 'condition': ['depression'],
 'review': ['"I\'ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia & anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I\'ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."'],
 'rating': [10.0],
 'date': ['February 28, 2012'],
 'usefulCount': [22],
 'review_length': [68]}

In [50]:
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [51]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

In [52]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 3min 40s, sys: 3.22 s, total: 3min 43s
Wall time: 1min 18s


In [60]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

In [61]:
def slow_tokenize_function(examples):
    return slow_tokenizer(examples["review"], truncation=True)

In [58]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

CPU times: user 57.3 ms, sys: 127 ms, total: 184 ms
Wall time: 479 ms


In [62]:
%time tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 6min 41s, sys: 5.06 s, total: 6min 47s
Wall time: 8min 43s


In [ ]:
%time tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=False)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]